In [ ]:
import pandas as pd
# df=pd.read_csv('/home/bram/bramenv/afeaturematrix/ALLHOSPITALS_DATA.csv')
df=pd.read_csv('/home/bram/bramenv/afeaturematrix/ALLHOSPITALS_NEW_DATA.csv')
print(len(df))
print(df.columns)
print(df['BDSPPatientID'].nunique())
print(len(df))
print(df.columns)
print(df)
import pandas as pd
import re
import nltk
from nltk.stem import SnowballStemmer
# Ensure you have the necessary nltk resources
nltk.download('punkt')
# Initialize the Snowball Stemmer for English
stemmer = SnowballStemmer("english")
# # Sample data
# data = {
#     "patientID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
#     "createdate": ["2023-01-01", "2023-01-02", "2023-01-03", "2023-01-04", "2023-01-05", "2023-01-06", "2023-01-07", "2023-01-08",
#                    "2023-01-09", "2023-01-10", "2023-01-11", "2023-01-12", "2023-01-12", "2023-01-12", "2023-01-12", "2023-01-12", "2023-01-12", "2023-01-12", "2023-01-12", "2023-01-12", "2023-01-12"],
#     "note": [
#         "There is no hypoxic injury observed. However, the patient did not show signs of hypoxic injury until later.",
#         "The report mentioned that the condition isn't indicative of hypoxic injury. There are no other injuries observed.",
#         "absent signs of neurological deficit were found. The patient didn't show any evidence of infection.",
#         "Signs of hypoxic injury were observed, but no other injuries were found.",
#         "The patient shows hypoxic injury and neurological deficit.",
#         "Initially, there was no evidence of infection, but later signs of infection were observed.",
#         "Patient exhibits signs of hypoxic injury.",
#         "There is no evidence of hypoxic injury or other injuries.",
#         "Neurological deficit is absent in the patient.",
#         "The patient showed neurological deficit initially, but it was resolved later.",
#         "Hypoxic injury is absent.",
#         "Other injuries are absent, but there is evidence of infection.",
#         'parkinsonism',
#         'sparkinsonism',
#         'parkinsonisms', 'no parkinsonism',
#         'sparkinsonism absent',
#         'not parkinsonisms',
#         'no parkinson',
#         '  parkinson',
#         '  parkinson absent'
#     ]
# }
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', None)
# # Create DataFrame
# df = pd.DataFrame(data)
# Define the keywords
# keywords = ["hypoxic injury", "other injuries", "neurological deficit", "evidence of infection", 'parkinson', 'parkinsonismx']
keywords = [
    "parkinson disease", "pd", "parkinson", 'parkinsonismx', "disease", "paralysis agitans", "primary parkinsonism",
    "pdd", "parkinsonism", "akathisia", "alien limb", "alpha synuclein", "anosmia", "anticholinergic side effect",
    "apraxia", "autonomic", "axial", "basophilic", "blepharospasm", "bradyphasia", "bradyphonia",
    "bradyphrenia", "camptocormia", "cholinergic", "cogwheel", "cognitive impairment", "corticobasal",
    "daws", "denervation", "dopamine", "dopamine agonist", "dysphagia", "dysphonia", "dystonia",
    "facial", "fasciculation", "gba", "gait", "festinating", "shuffling", "freezing", "globus",
    "hyposmia", "hyperreflexia", "hypokinetic", "hypomimia", "hypophonia", "idiopathic", "imaging",
    "dat spect", "datscan", "dat", "mri", "scan", "lrrk2", "snca", "prkn", "pink1", "dj-1", "gba",
    "vps35", "atp13a2", "lrrk1", "uchl1", "mao-b", "monoamine oxidase b inhibitor", "masked facies",
    "medullary", "micrographia", "mild cognitive impairment", "mci", "montreal cognitive assessment",
    "moca", "movement disorder", "multiple system atrophy", "msa", "nigra pars compacta", "nigrostriatal",
    "ocular torticollis", "occupational therapy", "orthostatic", "orofacial", "pallidotomy",
    "pallidothalamic", "palliative", "paratonia", "pathology", "pet", "physical therapy", "postencephalitic",
    "posture", "primary progressive aphasia", "ppa", "prodrome", "progressive supranuclear palsy",
    "psp", "pyramidal", "rem sleep behavior disorder", "rbd", "restless legs", "saccade", "sialorrhea",
    "speech therapy", "spasmodic", "stooped", "subthalamic nucleus", "stn", "striatal dopamine deficiency",
    "stiff", "rigidity", "subcortical", "supranuclear", "swallowing", "tetany", "thalamotomy", "tics",
    "tolcapone", "toronto cognitive assessment", "torca", "tremor", "uptake", "vascular", "nausea",
    "hallucination", "constipation", "fatigue", "insomnia", "depression", "anxiety", "psychosis",
    "dosage adjustment", "physiotherapy", "lifestyle changes", "support group", "caregiver",
    "medication adherence", "prescription refill", "fall precaution", "daily function", "self care",
    "quality of life", "disease progression", "regular follow-up", "exercise", "sleep hygiene",
    "bone health", "care coordination", "shuffling gait", "exelon patch", "exelon", "cane", "fall",
    "deep brain stimulation", "dbs", "bradykinesia", "hypokinesia", "dementia", "pulmonary complication",
    "pneumonia"
]
# Function to stem each word in a phrase
def stem_phrase(phrase, stemmer):
    tokens = nltk.word_tokenize(phrase)
    return ' '.join([stemmer.stem(token) for token in tokens])
# Generate regex patterns
def generate_patterns(keywords):
    neg_patterns = {}
    pos_patterns = {}
    for phrase in keywords:
        stemmed_phrase = stem_phrase(phrase, stemmer)
        words = stemmed_phrase.split()
        neg_pattern = r"\b" + r"(?:(?:no|not|n't|absent|negative)[^,.]{{0,30}}{pattern})|(?:{pattern}[^.]{{0,80}}(?:absent|negative))".format(
            pattern= r''.join([f"{word}\s*" for word in words]) +r"\b"
        )
        neg_patterns[phrase] = neg_pattern
        pos_patterns[phrase] = r"\b" + r"".join([f"{word}\s*" for word in words]) +r"\b"
    return neg_patterns, pos_patterns
neg_patterns, pos_patterns = generate_patterns(keywords)
# Print generated patterns for verification
print("Negated Patterns:")
for key, pattern in neg_patterns.items():
    print(f"{key}: {pattern}")
print("\nPositive Patterns:")
for key, pattern in pos_patterns.items():
    print(f"{key}: {pattern}")
# Function to preprocess text (lowercase, stemming using NLTK)
def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r"\b[a-z]*parkinsonism[a-z]*\b", "parkinsonismx", text)  # Replace words containing 'parkinsonism'
    # print (text)
    tokens = nltk.word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)
# Function to check for the presence of patterns and determine the final occurrence
def check_patterns(text, neg_patterns, pos_patterns):
    # print(text)
    matches = {key: 0 for key in set(neg_patterns) | set(pos_patterns)}  # Initialize matches dictionary with 0
    for key in matches:
        neg_match = [m.end() for m in re.finditer(neg_patterns[key], text)]
        pos_match = [m.end() for m in re.finditer(pos_patterns[key], text)]
        # # Debugging information
        # print(f"\nProcessing key: {key}")
        # print(f"Text: {text}")
        # print(f"Negated matches end positions: {neg_match}")
        # print(f"Positive matches end positions: {pos_match}")
        # Determine the latest match between negated and non-negated patterns
        latest_neg_match = max(neg_match, default=-1)
        latest_pos_match = max(pos_match, default=-1)
        print(f"Latest negated match end position: {latest_neg_match}")
        print(f"Latest positive match end position: {latest_pos_match}")
        # Final assignment logic
        if latest_neg_match != -1 and (latest_pos_match == -1 or latest_neg_match >= latest_pos_match):
            matches[key] = -1
        elif latest_pos_match != -1:
            matches[key] = 1
        # Debugging information
        print(f"Assigned value for {key}: {matches[key]}")
    return matches
# Apply preprocessing and pattern checking to each note
df['processed_note'] = df['note_text'].apply(preprocess_text)
# Display the preprocessed notes for verification
print("Preprocessed Notes:")
print(df['processed_note'])
pattern_features = df['processed_note'].apply(lambda note_text: check_patterns(note_text, neg_patterns, pos_patterns))
# Convert the pattern features to a DataFrame
pattern_columns = pd.DataFrame(pattern_features.tolist())
# Concatenate with the original DataFrame
result = pd.concat([df[['BDSPPatientID', 'CreateDate', 'hospital']], pattern_columns], axis=1)
# Display the result
print(result)
result.to_csv('FEATMATRIX_TEXT.csv', index=False)